In [1]:
import requests
from bs4 import BeautifulSoup as bs
import types
import json
import pymysql

owner_id="4247543"
user_url = "https://stackoverflow.com/users/" + owner_id
user_page = bs(requests.get(user_url).text,'html.parser')
    
reputation = user_page.find('div',class_='reputation').text.split(' r')[0]#可能大於一千，就會有,不能直接轉型
reputation = int(reputation.strip().replace(',',''))
print(reputation)

times=0
sql_text = ""
for x in user_page.find_all('a',class_='post-tag'):
    sql_text += x.text + "|"   
    for y in user_page.find_all('span',class_='g-col -number'):
        times+=1
        if(times==1 or times==6 or times==13 or times==22 or times==33 or times==46):#score
            sql_text += y.text + ',' 
        if(times==2 or times==7 or times==14 or times==23 or times==34 or times==47):#post
            sql_text += y.text + " , "
            break
            
print(sql_text)

1536
android|85,150 , firebase|20,9 , android-studio|11,11 , xml|9,7 , android-fragments|6,21 , java|5,40 , 


In [8]:
answers = int(user_page.find('div',class_='stat answers col-3').text.split('a')[0].strip())
print(answers)

questions = int(user_page.find('div',class_='stat questions col-3').text.split('q')[0].strip())
print(questions)

reached = '"' + user_page.find('div',class_='stat people-helped col-5').text.split('p')[0].strip() + '"'
print(reached)

#以上是頁面右上角的三個東西，其中answers和questions型態是int

profile = user_page.find('ul',class_='list-unstyled')
times = 0
for x in profile.find_all('li'):
    if(times == 0):
        location = '"' + x.text.strip() + '"'
    elif(times == 1):
        age = '"' + x.text.strip() + '"'
    else:
        views = int(x.text.split(' p')[0].strip().replace(',',''))
    times += 1 
    if(times == 3):#times是上次登入時間，不需要抓下來，直接break避免再find_all一次
        break
print(location + ' | ' + age + ' | ' , views)

151
5
"~131k"
"Ernakulam, Kerala, India" | "Member for 3 years, 4 months" |  199


In [3]:
toptags_url = 'https://api.stackexchange.com/2.2/users/' + owner_id + '/top-tags?site=stackoverflow'
user_toptags = bs(requests.get(toptags_url).text,'html.parser')


json_str = json.loads(user_toptags.text)

total_count = 0
total_score = 0
for x in range(len(json_str['items'])):
    total_count = json_str['items'][x]['answer_count'] + json_str['items'][x]['question_count']
    total_score = json_str['items'][x]['answer_score'] + json_str['items'][x]['question_score']
    print('answer_count: ',json_str['items'][x]['answer_count'],'answer_score: ',json_str['items'][x]['answer_score'],
         'question_count: ',json_str['items'][x]['question_count'],'question_score: ',json_str['items'][x]['question_score'],
         'total_count:',total_count,'total_score: ',total_score,'tag_name: ',json_str['items'][x]['tag_name'])

answer_count:  146 answer_score:  85 question_count:  4 question_score:  7 total_count: 150 total_score:  92 tag_name:  android
answer_count:  9 answer_score:  20 question_count:  0 question_score:  0 total_count: 9 total_score:  20 tag_name:  firebase
answer_count:  3 answer_score:  12 question_count:  0 question_score:  0 total_count: 3 total_score:  12 tag_name:  firebase-cloud-messaging
answer_count:  11 answer_score:  11 question_count:  0 question_score:  0 total_count: 11 total_score:  11 tag_name:  android-studio
answer_count:  38 answer_score:  5 question_count:  2 question_score:  4 total_count: 40 total_score:  9 tag_name:  java
answer_count:  7 answer_score:  9 question_count:  0 question_score:  0 total_count: 7 total_score:  9 tag_name:  xml
answer_count:  3 answer_score:  8 question_count:  0 question_score:  0 total_count: 3 total_score:  8 tag_name:  android-studio-2.2
answer_count:  1 answer_score:  8 question_count:  0 question_score:  0 total_count: 1 total_score:  

In [13]:
db = pymysql.connect('140.138.77.90','knkn','tp6bjo4u;6','knkn',use_unicode=True, charset="utf8mb4")
cursor = db.cursor()
cursor.execute('INSERT INTO user (user_id,reputation,answers,questions,reached,location,age,views) VALUES('
               +owner_id+','+str(reputation)+','+str(answers)+','+str(questions)+','+reached+','+location+','+age+','+str(views)+')')

db.commit()
cursor.close()
db.close()